In [1]:
import numpy as np
import cv2
cam=cv2.VideoCapture(0)

# instantiate a camera object to capture images

face_cas=cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
#declare type of font
font=cv2.FONT_HERSHEY_SIMPLEX
f_01=np.load('face_02.npy').reshape((40,50*50*3))


# load the data from the numpy matrices and convert to linear vectors


print(f_01.shape)
#print(f_01)
# create a look-up dictionary
names={0: 'Guri'}
labels=np.zeros((40,1))
labels[:40,:]=0.0
data=np.concatenate([f_01])
print(labels.shape,data.shape)
#there are 20 matrixes and each of shap is 50*50*3

(40, 7500)
(40, 1) (40, 7500)


In [2]:
# the distance and knn functions 
def distance(x1, x2):
    return np.sqrt(((x1-x2)**2).sum())

def knn(x, train, targets, k=5):
    m = train.shape[0]
    dist = []
    for ix in range(m):
        # compute distance from each point and store in dist
        dist.append(distance(x, train[ix]))
    dist = np.asarray(dist)
    indx = np.argsort(dist)
    sorted_labels = labels[indx][:k]
    counts = np.unique(sorted_labels, return_counts=True)
    return counts[0][np.argmax(counts[1])]


In [6]:
while True:
	# get each frame
	ret, frame = cam.read()

	if ret == True:
		# convert to grayscale and get faces
		gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		faces = face_cas.detectMultiScale(gray, 1.3, 5)

		# for each face
		for (x, y, w, h) in faces:
			face_component = frame[y:y+h, x:x+w, :]
			fc = cv2.resize(face_component, (50, 50))

			# after processing the image and rescaling
			# convert to linear vector using .flatten()
			# and pass to knn function along with all the data

			lab = knn(fc.flatten(), data, labels)
			# convert this label to int and get the corresponding name
			text = names[int(lab)]

			# display the name
			cv2.putText(frame, text, (x, y-9), font, 1, (0, 255,255), 2)

			# draw a rectangle over the face
			cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 0, 255), 2)
		cv2.imshow('face recognition', frame)

		if cv2.waitKey(1) == 27:
			break
	else:
		print ('Error')

cv2.destroyAllWindows()
# Some times of lightning and distance from camera it shows wrong name
# To improve that we should take more data

        
            